In [2]:
import os
import cv2
import numpy as np
import mediapipe as mp
from sklearn.model_selection import train_test_split

mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)
pose = mp_pose.Pose(min_detection_confidence=0.7)

# Define 50 signs
SIGNS = [
    'One', 'Great','Young','Open','Strong','Water'
]

DATASET_PATH = 'dataset'
FRAMES_PER_VIDEO = 30
EXPECTED_LANDMARKS = 225  # 21*3*2 + 33*3
OUTPUT_PATH = 'processed_data'

def extract_landmarks(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    hand_results = hands.process(image_rgb)
    pose_results = pose.process(image_rgb)
    
    landmarks = []
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks[:2]:
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])
        landmarks.extend([0] * (21 * 3 * (2 - len(hand_results.multi_hand_landmarks))))
    else:
        landmarks.extend([0] * (21 * 3 * 2))
    
    if pose_results.pose_landmarks:
        for i, lm in enumerate(pose_results.pose_landmarks.landmark):
            if i < 33:
                landmarks.extend([lm.x, lm.y, lm.z])
    else:
        landmarks.extend([0] * (33 * 3))
    
    landmarks = np.array(landmarks)
    if len(landmarks) != EXPECTED_LANDMARKS:
        if len(landmarks) < EXPECTED_LANDMARKS:
            landmarks = np.pad(landmarks, (0, EXPECTED_LANDMARKS - len(landmarks)), mode='constant')
        else:
            landmarks = landmarks[:EXPECTED_LANDMARKS]
    
    return landmarks

def preprocess_data():
    X, y = [], []
    label_map = {sign: idx for idx, sign in enumerate(SIGNS)}
    
    for sign in SIGNS:
        sign_path = os.path.join(DATASET_PATH, sign)
        if not os.path.exists(sign_path):
            print(f"Warning: Sign directory {sign_path} not found")
            continue
        
        for video_idx in range(10):  # Assuming 10 videos per sign
            video_path = os.path.join(sign_path, f"video_{video_idx}")
            if not os.path.exists(video_path):
                print(f"Warning: Video directory {video_path} not found")
                continue
            
            frames = []
            for frame_idx in range(FRAMES_PER_VIDEO):
                frame_path = os.path.join(video_path, f"frame_{frame_idx}.jpg")
                if not os.path.exists(frame_path):
                    print(f"Warning: Frame {frame_path} not found")
                    break
                frame = cv2.imread(frame_path)
                landmarks = extract_landmarks(frame)
                frames.append(landmarks)
            
            if len(frames) == FRAMES_PER_VIDEO:
                X.append(frames)
                y.append(label_map[sign])
            else:
                print(f"Warning: Incomplete video {video_path}, found {len(frames)} frames")
    
    if not X:
        print("Error: No valid data found. Check dataset.")
        return
    
    X = np.array(X)  # Shape: [num_videos, FRAMES_PER_VIDEO, EXPECTED_LANDMARKS]
    y = np.array(y)  # Shape: [num_videos]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    np.save(os.path.join(OUTPUT_PATH, 'X_train.npy'), X_train)
    np.save(os.path.join(OUTPUT_PATH, 'X_test.npy'), X_test)
    np.save(os.path.join(OUTPUT_PATH, 'y_train.npy'), y_train)
    np.save(os.path.join(OUTPUT_PATH, 'y_test.npy'), y_test)
    np.save(os.path.join(OUTPUT_PATH, 'labels.npy'), np.array(SIGNS))
    
    print(f"Processed data saved: {X.shape[0]} videos, {len(SIGNS)} signs")
    print(f"Train: {X_train.shape[0]} videos, Test: {X_test.shape[0]} videos")

if __name__ == "__main__":
    preprocess_data()
    hands.close()
    pose.close()

Processed data saved: 60 videos, 6 signs
Train: 48 videos, Test: 12 videos


In [ ]:
SIGNS = [
    'Hello', 'Great', 'Sunny', 'Brave', 'Kind', 'Happy', 'Strong', 'Wise', 'Gentle', 'Bright',
    'Calm', 'Quick', 'Warm', 'Clear', 'Bold', 'Sweet', 'Pure', 'Vivid', 'Steady', 'Lively',
    'Soft', 'True', 'Deep', 'Light', 'Fast', 'Cool', 'High', 'Smooth', 'Rich', 'Quiet',
    'Firm', 'New', 'Old', 'Young', 'Safe', 'Wild', 'Hot', 'Cold', 'Dry', 'Wet',
    'Early', 'Late', 'Near', 'Far', 'Full', 'Empty', 'Sharp', 'Dull', 'Loud', 'I'
]
